In [1]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 749 kB 22.8 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 27.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import nltk

In [5]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('data/amazon_NLP_data.csv' , encoding='utf-8-sig')

In [16]:
df['review'] = df['review'].str.replace("[^a-zA-Z ]","")
df['review'].replace('', np.nan, inplace=True)
df = df.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(df))

# numpy select

df['label'] = np.select([df.rating > 3], [1], default=0)


전처리 후 테스트용 샘플의 개수 : 877


In [19]:
df['tokenized'] = df['review'].apply(nltk.word_tokenize)

In [21]:
df = df[df['rating']!= 3]

In [23]:
df.rating.value_counts()

5    524
4    106
1    105
2     60
Name: rating, dtype: int64

In [43]:
pos_series = df[df['label'] == 1].tokenized
neg_series = df[df['label'] == 0].tokenized

In [47]:
pos_list = [element for list_ in pos_series.values for element in list_]
neg_list = [element for list_ in neg_series.values for element in list_]

In [40]:
import collections

In [45]:
collections.Counter(pos_list).most_common()

[('I', 516),
 ('and', 475),
 ('my', 369),
 ('it', 364),
 ('the', 346),
 ('a', 331),
 ('this', 301),
 ('primer', 271),
 ('to', 268),
 ('on', 215),
 ('for', 190),
 ('is', 181),
 ('makeup', 160),
 ('skin', 159),
 ('product', 141),
 ('of', 127),
 ('so', 127),
 ('smooth', 114),
 ('but', 111),
 ('have', 108),
 ('foundation', 108),
 ('like', 107),
 ('as', 104),
 ('great', 101),
 ('It', 91),
 ('love', 87),
 ('with', 87),
 ('This', 84),
 ('in', 83),
 ('face', 79),
 ('that', 78),
 ('all', 73),
 ('use', 73),
 ('well', 68),
 ('up', 67),
 ('good', 66),
 ('not', 65),
 ('day', 61),
 ('goes', 60),
 ('price', 60),
 ('Great', 60),
 ('make', 59),
 ('was', 59),
 ('very', 58),
 ('out', 56),
 ('much', 55),
 ('Love', 54),
 ('used', 54),
 ('look', 53),
 ('makes', 52),
 ('its', 52),
 ('me', 52),
 ('Ive', 51),
 ('works', 50),
 ('just', 50),
 ('really', 49),
 ('your', 48),
 ('does', 46),
 ('Im', 43),
 ('will', 43),
 ('be', 43),
 ('My', 42),
 ('more', 41),
 ('you', 41),
 ('oily', 41),
 ('would', 39),
 ('Its', 36)

In [46]:
collections.Counter(neg_list).most_common()

[('I', 171),
 ('it', 158),
 ('the', 133),
 ('and', 123),
 ('my', 105),
 ('to', 91),
 ('not', 86),
 ('a', 84),
 ('this', 80),
 ('is', 70),
 ('product', 67),
 ('primer', 53),
 ('was', 52),
 ('redness', 49),
 ('of', 47),
 ('for', 45),
 ('skin', 45),
 ('but', 43),
 ('on', 40),
 ('in', 37),
 ('does', 36),
 ('It', 33),
 ('at', 33),
 ('This', 33),
 ('face', 32),
 ('like', 30),
 ('with', 30),
 ('cover', 28),
 ('all', 27),
 ('or', 27),
 ('as', 27),
 ('very', 24),
 ('did', 23),
 ('red', 22),
 ('you', 22),
 ('green', 21),
 ('that', 21),
 ('me', 21),
 ('foundation', 21),
 ('have', 21),
 ('nothing', 21),
 ('oily', 20),
 ('so', 20),
 ('money', 20),
 ('doesnt', 20),
 ('if', 20),
 ('work', 19),
 ('good', 19),
 ('just', 19),
 ('out', 18),
 ('after', 18),
 ('didnt', 17),
 ('more', 17),
 ('help', 17),
 ('use', 17),
 ('off', 16),
 ('Im', 16),
 ('no', 15),
 ('when', 15),
 ('one', 14),
 ('really', 14),
 ('The', 14),
 ('feel', 14),
 ('any', 14),
 ('even', 14),
 ('anything', 14),
 ('would', 14),
 ('its', 13),